In [46]:
import pandas as pd
import numpy as np
import pickle as p

In [72]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=5, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [73]:
# item_subset

### `/train`

In [90]:
import requests
import json

url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
# data = data_subset.to_json()

files = {'data':open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True,
           'project_id': 18
           }

headers = {
           'Accept-Charset': 'UTF-8'
          }

# payload = json.dumps(payload)
r = requests.post(url, data=payload,  timeout=1200, files=files)

files['data'].close()

In [92]:
from threading import Thread
import requests
import json

files = {'data':open('data_subset.parquet', 'rb')}
Thread(target=requests.post, args=(url,), kwargs={'data': payload, 'files':files}).start()

In [79]:
r.json()['result']

'Success'

In [80]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

,avg_sales,item_id,mae_score,mpe_score,r2_score,rmse_score
0,2110.440186,202.0,435.395748,0.206306,0.594991,584.987244
1,109.613182,16019.0,32.700777,0.298329,0.181786,41.813029
2,91.448227,3766.0,21.738481,0.237714,0.537483,29.427667
3,126.142509,3732.0,26.777396,0.212279,0.637215,35.385051
4,8.996674,1230.0,4.258772,0.473372,0.265836,5.777962


### `/query_progress`

In [6]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'


payload = {'project_id': 1}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 80.0
CV Done: 0


### `/get_cv_results`

In [95]:
import requests
import json

url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 18}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=1200)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,1230.0,8.996674,0.220607,4.401550,0.489242,6.058230
1,3766.0,91.448227,0.510286,24.574961,0.268731,33.884080
2,3732.0,126.142509,0.452335,30.020367,0.237988,38.960951
3,16019.0,109.613182,0.387819,36.506746,0.333051,47.586530
4,202.0,2110.440186,0.568045,443.745052,0.210262,587.911751


### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [49]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'202': 2.35,
 '1195': 3.15,
 '1230': 3.25,
 '2531': 6.25,
 '2589': 11.9,
 '3726': 9.95,
 '3732': 11.55,
 '3743': 11.1,
 '3766': 9.55,
 '16019': 13.6}

In [50]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 12}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_1195,223.0,195.0
Qty_1230,19.0,17.0
Qty_16019,210.0,210.0
Qty_202,5193.0,4785.0
Qty_2531,517.0,703.0
Qty_2589,419.0,277.0
Qty_3726,243.0,254.0
Qty_3732,342.0,350.0
Qty_3743,210.0,242.0
Qty_3766,241.0,227.0
